In [3]:
!pip install openai

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI  # Importa a biblioteca da OpenAI
from google.colab import userdata

# --- CONFIGURAÇÃO DA IA (AGORA COM OPENROUTER) ---
try:
    # Acessa a chave de API guardada no 'Secrets' do Colab
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    print("API Key do OpenRouter encontrada.")

    # Inicializa o cliente da OpenAI, apontando para a URL do OpenRouter
    client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key=OPENROUTER_API_KEY,
    )
    print("Cliente configurado para usar OpenRouter.")

except Exception as e:
    print("Erro ao configurar a API. Verifique se você instalou a biblioteca openai e configurou o Secret 'OPENROUTER_API_KEY'.")
    print(e)
    exit()

# --- CARREGAMENTO E PREPARAÇÃO DOS DADOS (sem alterações) ---

try:
    df = pd.read_csv('banco.csv')
    print("Arquivo 'banco.csv' carregado.")
except FileNotFoundError:
    print("Erro: Arquivo 'banco.csv' não encontrado. Faça o upload do arquivo no Colab.")
    exit()

df['data'] = pd.to_datetime(df['data'])

def define_setor(loja):
    lojas_varejo = ['Constrular Materiais', 'Reforma Total Depósito', 'Obra&Cia']
    if loja in lojas_varejo:
        return 'Varejo de Construção'
    else:
        return 'Hotelaria'

df['setor'] = df['local/loja'].apply(define_setor)
print("Dados processados. Iniciando análises...")

print("Calculando métricas balanceadas para a IA...")
# Métricas gerais
sentimento_por_setor = df.groupby('setor')['sentimento'].value_counts().unstack(fill_value=0)
urgencia_counts = df['urgencia'].value_counts().reindex(['Baixa', 'Média', 'Alta', 'Crítica'])
geral_categorias = df['categoria'].value_counts().nlargest(10) # Assuntos mais comentados no geral

# Métricas de Pontos Fortes (POSITIVOS)
df_positivo = df[df['sentimento'] == 'Positivo']
top_categorias_positivas = df_positivo['categoria'].value_counts().nlargest(5)

# Métricas de Pontos Fracos (NEGATIVOS)
df_negativo = df[df['sentimento'] == 'Negativo']
top_categorias_negativas = df_negativo['categoria'].value_counts().nlargest(5)


# --- IA PARA RESUMO EXECUTIVO (COM PROMPT ATUALIZADO) ---

print("Conectando com a IA para gerar um novo resumo estratégico...")

system_prompt = "Você é um analista de negócios sênior, especialista em experiência do cliente. Sua análise deve ser equilibrada, destacando tanto os pontos fortes quanto as oportunidades de melhoria."
user_prompt = f"""
Analise os seguintes dados agregados de feedback de clientes para uma empresa com operações em Hotelaria e Varejo de Construção. Forneça um relatório justo e balanceado.

**DADOS AGREGADOS:**

1.  **Visão Geral do Sentimento por Setor:**
{sentimento_por_setor.to_string()}

2.  **Assuntos Mais Comentados (Geral):**
{geral_categorias.to_string()}

3.  **PRINCIPAIS PONTOS FORTES (Categorias Mais Elogiadas):**
{top_categorias_positivas.to_string()}

4.  **PRINCIPAIS PONTOS FRACOS (Categorias Mais Criticadas):**
{top_categorias_negativas.to_string()}

5.  **Níveis de Urgência dos Feedbacks:**
{urgencia_counts.to_string()}

**SUA TAREFA:**

Com base nos dados, escreva um resumo executivo equilibrado:
1.  Comece com um parágrafo de visão geral.
2.  Identifique os **2 principais PONTOS FORTES** que devem ser celebrados e reforçados.
3.  Identifique os **2 principais PONTOS FRACOS** ou de maior risco que exigem ação imediata.
4.  Forneça uma **conclusão geral com uma recomendação estratégica** para a empresa.
"""

try:
    response = client.chat.completions.create(
      model="meta-llama/llama-3.1-8b-instruct",
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
      ],
      temperature=0.6,
    )

    print("\n\n" + "="*80)
    print("✅ RESUMO EXECUTIVO BALANCEADO GERADO PELA IA")
    print("="*80 + "\n")
    print(response.choices[0].message.content)
    print("\n" + "="*80)

except Exception as e:
    print(f"\nOcorreu um erro ao gerar o relatório com a IA: {e}")

API Key do OpenRouter encontrada.
Cliente configurado para usar OpenRouter.
Arquivo 'banco.csv' carregado.
Dados processados. Iniciando análises...
Calculando métricas balanceadas para a IA...
Conectando com a IA para gerar um novo resumo estratégico...


✅ RESUMO EXECUTIVO BALANCEADO GERADO PELA IA

**Resumo Executivo:**

A análise dos dados agregados de feedback de clientes revela uma visão geral mista dos dois setores da empresa, Hotelaria e Varejo de Construção, com uma tendência levemente negativa. Os clientes mencionam uma variedade de categorias como principais pontos de atenção, refletindo as complexidades da experiência de cliente em diferentes áreas. No entanto, há oportunidades claras para aprimoramento.

**Pontos Fortes:**

Os pontos fortes da empresa estão relacionados ao atendimento, localização e serviços oferecidos. O atendimento recebeu destaque como a categoria mais elogiada, destacando a importância da qualidade do tratamento dispensado aos clientes. A localização ta